# calculate rotation and transposition matrix

In [2]:
data_folder = r'D:\Shiwei\20210423-E14_Forebrain_smRNA'
before_position_file = os.path.join(data_folder, '10x_positions.txt')
after_position_file = os.path.join(data_folder, '10x_positions_after.txt')


In [1]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [3]:
R, T = align_manual_points(before_position_file, after_position_file, save=False)


- Manually picked points aligned, rotation:
[[ 0.99989683 -0.01436443]
 [ 0.01436443  0.99989683]],
 translation:[ 593.98776915 -207.2862716 ]


In [ ]:
R, T

# transpose 60x positions

In [4]:
old_positions = np.loadtxt(os.path.join(data_folder, 'positions_all.txt'), delimiter=',')

In [5]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-3331.05751255 -4073.80416197]
 [-3328.19755482 -3874.72470385]
 [-3325.33759709 -3675.64524574]
 [-3322.4790758  -3476.66577732]
 [-3319.61911806 -3277.5863192 ]
 [-3316.75916033 -3078.50686109]
 [-3117.7796919  -3081.36538238]
 [-3120.63964963 -3280.4448405 ]
 [-3123.49960737 -3479.52429861]
 [-3126.35812866 -3678.50376703]
 [-3129.21808639 -3877.58322515]
 [-3132.07804413 -4076.66268326]
 [-2932.99858602 -4079.52264099]
 [-2930.13862828 -3880.44318288]
 [-2927.27867055 -3681.36372477]
 [-2924.42014926 -3482.38425634]
 [-2921.56019152 -3283.30479823]
 [-2918.70023379 -3084.22534012]
 [-2719.62077568 -3087.08529785]
 [-2722.48073341 -3286.16475596]
 [-2725.34069115 -3485.24421407]
 [-2728.19921244 -3684.2236825 ]
 [-2731.05917017 -3883.30314061]
 [-2733.9191279  -4082.38259872]
 [-2534.93965948 -4085.24112001]
 [-2532.07970174 -3886.1616619 ]
 [-2529.21974401 -3687.08220379]
 [-2526.36122272 -3488.10273536]
 [-2523.50126498 -3289.02327725]
 [-2520.64130725 -3089.94381914]
 [-2321.56

In [6]:
save_filename = os.path.join(data_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

\\storm6-pc\STORM6-FLASHDrive\Pu\20201127-NOAcr_CTP-08_E14_brain_no_clearing\translated_positions_all.txt


# further adjust manually

In [8]:
manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [9]:
adj_save_filename = os.path.join(data_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

\\storm6-pc\STORM6-FLASHDrive\Pu\20201127-NOAcr_CTP-08_E14_brain_no_clearing\adjusted_translated_positions_all.txt
